extract phone seqs from MFA alignments and save in format correct for
- speechbrain ASR training
- fastpitch TTS training

# imports

In [59]:
import os
import tgt 
from tqdm import tqdm

In [60]:
# print hostname to make sure we are on correct node
import socket
print(socket.gethostname())

kinloch.inf.ed.ac.uk


# global vars 

In [61]:
ALIGNMENT_DIR = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/LJSpeech-1.1/MFA_alignments_lowercase_nopunc'

# load alignments

In [62]:
# sanity check - assert that each utt has a corresponding alignment
alignment_files = set(os.listdir(ALIGNMENT_DIR))

In [63]:
uttid2textgrid = {}
for textgrid in tqdm(alignment_files):
    utt_id = textgrid.split('.')[0]
    tg_path = os.path.join(ALIGNMENT_DIR, textgrid)
    tg_obj = tgt.io.read_textgrid(tg_path, include_empty_intervals=True)
    uttid2textgrid[utt_id] = tg_obj

100%|████████████████████████████████████████████████████████████| 13100/13100 [00:29<00:00, 439.03it/s]


# parse alignments to extract phone seq

In [64]:
def get_phones_per_word(tg, ignore_sil=True, sil_labels=['sil', 'sp', '']):
    phones, words = None, None
    phones_per_word = []
    for tier in tg.tiers:
        if tier.name == 'phones':
            phones = tier
        elif tier.name == 'words':
            words = tier
    if phones is None or words is None:
        raise ValueError('TextGrid does not contain phones or words tier')
    for word in words.annotations:
        if ignore_sil and word.text in sil_labels:
            continue
        word_start = word.start_time
        word_end = word.end_time
        word_text = word.text
        phoneseq = []
        for phone in phones.annotations:
            if phone.start_time >= word_start and phone.end_time <= word_end:
                phoneseq.append(phone.text)
        phones_per_word.append((word_text, phoneseq))
        ### DEBUG
        if 'spn' in phoneseq:
            print('spn in phoneseq', word, phoneseq)
    return phones_per_word

In [65]:
# sanity check
get_phones_per_word(uttid2textgrid['LJ001-0001'])

[('printing', ['p', 'ɹ', 'ɪ', 'n', 't', 'ɪ', 'ŋ']),
 ('in', ['ɪ', 'n']),
 ('the', ['ð', 'i']),
 ('only', ['ow', 'ɲ', 'ʎ', 'i']),
 ('sense', ['s', 'ɛ', 'n', 's']),
 ('with', ['w', 'ɪ', 'θ']),
 ('which', ['w', 'ɪ', 'tʃ']),
 ('we', ['w', 'iː']),
 ('are', ['ɑ', 'ɹ']),
 ('at', ['æ', 'ʔ']),
 ('present', ['p', 'ɹ', 'ɛ', 'z', 'ə', 'n', 't']),
 ('concerned', ['kʰ', 'ə', 'n', 's', 'ɝ', 'n', 'd']),
 ('differs', ['d', 'ɪ', 'f', 'ɚ', 'z']),
 ('from', ['f', 'ɹ', 'ɐ', 'm']),
 ('most', ['m', 'ow', 's', 't']),
 ('if', ['ɪ', 'f']),
 ('not', ['n', 'ɑ', 'ʔ']),
 ('from', ['f', 'ɹ', 'ɐ', 'm']),
 ('all', ['ɒː', 'ɫ']),
 ('the', ['ð', 'i']),
 ('arts', ['ɑ', 'ɹ', 't', 's']),
 ('and', ['æ', 'n']),
 ('crafts', ['c', 'ɹ', 'æ', 'f', 't', 's']),
 ('represented', ['ɹ', 'ɛ', 'p', 'ɹ', 'ɪ', 'z', 'ɛ', 'n', 't', 'ɪ', 'd']),
 ('in', ['ɪ', 'n']),
 ('the', ['ð', 'i']),
 ('exhibition', ['ɛ', 'k', 's', 'ɪ', 'b', 'ɪ', 'ʃ', 'ə', 'n'])]

In [66]:
uttid2phones_per_word = {}
mfa_symbol_set = set()
for uttid, textgrid in tqdm(uttid2textgrid.items()):
    phones_per_word = get_phones_per_word(textgrid)
    uttid2phones_per_word[uttid] = phones_per_word
    for word, phones in phones_per_word:
        mfa_symbol_set.update(phones)

  0%|▏                                                              | 31/13100 [00:00<00:43, 303.82it/s]

spn in phoneseq Interval(1.49, 2.18, "hideel") ['spn']
spn in phoneseq Interval(0.29, 0.79, "carrico") ['spn']
spn in phoneseq Interval(0.0, 0.04, "esarhaddon") ['spn']
spn in phoneseq Interval(0.41, 0.89, "sinacherib") ['spn']
spn in phoneseq Interval(8.12, 8.73, "oswalds") ['spn']
spn in phoneseq Interval(1.09, 1.66, "punchmark") ['spn']
spn in phoneseq Interval(0.0, 0.41, "roupell") ['spn']
spn in phoneseq Interval(0.0, 0.6, "scoggins") ['spn']
spn in phoneseq Interval(2.17, 2.62, "eking") ['spn']
spn in phoneseq Interval(0.55, 1.17, "askern") ['spn']
spn in phoneseq Interval(0.35, 1.01, "courvoisier") ['spn']


  1%|▍                                                              | 93/13100 [00:00<00:43, 302.35it/s]

spn in phoneseq Interval(2.57, 3.39, "prs") ['spn']
spn in phoneseq Interval(2.1, 2.86, "strychnia") ['spn']
spn in phoneseq Interval(0.15, 1.09, "ineffectively") ['spn']
spn in phoneseq Interval(5.2, 5.92, "bringuier") ['spn']
spn in phoneseq Interval(1.68, 2.05, "misess") ['spn']
spn in phoneseq Interval(6.25, 6.91, "millbank") ['spn']
spn in phoneseq Interval(6.14, 6.82, "connally") ['spn']
spn in phoneseq Interval(0.83, 1.25, "euins") ['spn']
spn in phoneseq Interval(0.95, 1.67, "cadigan") ['spn']
spn in phoneseq Interval(0.0, 0.52, "maltby") ['spn']
spn in phoneseq Interval(8.2, 9.09, "prs") ['spn']
spn in phoneseq Interval(1.86, 2.57, "prs") ['spn']
spn in phoneseq Interval(1.04, 1.85, "sandersons") ['spn']
spn in phoneseq Interval(7.25, 8.03, "contractile") ['spn']
spn in phoneseq Interval(4.38, 4.79, "misess") ['spn']
spn in phoneseq Interval(5.39, 5.89, "murret") ['spn']
spn in phoneseq Interval(0.16, 0.73, "quigleys") ['spn']
spn in phoneseq Interval(1.71, 2.23, "whitneys") [

  1%|▊                                                             | 163/13100 [00:00<00:39, 327.28it/s]

...

spn in phoneseq Interval(0.0, 0.67, "prs") ['spn']
spn in phoneseq Interval(2.79, 3.47, "shrillness") ['spn']
spn in phoneseq Interval(2.01, 2.46, "newmans") ['spn']
spn in phoneseq Interval(5.33, 5.78, "hosty") ['spn']
spn in phoneseq Interval(4.26, 4.37, "wouldn") ['spn']
spn in phoneseq Interval(0.0, 0.39, "misess") ['spn']
spn in phoneseq Interval(4.06, 4.85, "caslon") ['spn']
spn in phoneseq Interval(0.66, 1.27, "lovelady") ['spn']
spn in phoneseq Interval(0.71, 1.33, "sorrels") ['spn']
spn in phoneseq Interval(6.0, 6.84, "bringuier") ['spn']
spn in phoneseq Interval(6.64, 7.23, "wakefields") ['spn']
spn in phoneseq Interval(4.53, 5.26, "snatchers") ['spn']
spn in phoneseq Interval(0.0, 0.75, "zopyrus") ['spn']


100%|███████████████████████████████████████████████████████████▋| 13045/13100 [00:43<00:00, 314.61it/s]

spn in phoneseq Interval(7.69, 8.51, "laverstock") ['spn']
spn in phoneseq Interval(0.0, 0.64, "scoggins") ['spn']
spn in phoneseq Interval(2.28, 3.05, "nutfield") ['spn']
spn in phoneseq Interval(3.61, 4.32, "reigate") ['spn']
spn in phoneseq Interval(2.91, 3.55, "oswalds") ['spn']
spn in phoneseq Interval(0.0, 0.4, "hosty") ['spn']
spn in phoneseq Interval(1.25, 2.02, "roominghouse") ['spn']
spn in phoneseq Interval(3.72, 4.79, "resurrectionist") ['spn']
spn in phoneseq Interval(0.54, 1.63, "familiarization") ['spn']
spn in phoneseq Interval(1.01, 1.68, "stombaughs") ['spn']
spn in phoneseq Interval(2.83, 3.33, "probative") ['spn']
spn in phoneseq Interval(0.73, 1.28, "youngblood") ['spn']
spn in phoneseq Interval(3.52, 4.22, "greenacre") ['spn']
spn in phoneseq Interval(8.65, 9.78, "courvoisiers") ['spn']
spn in phoneseq Interval(0.33, 0.81, "paines") ['spn']
spn in phoneseq Interval(6.01, 6.47, "decamped") ['spn']
spn in phoneseq Interval(1.93, 2.62, "christs") ['spn']
spn in phone

100%|████████████████████████████████████████████████████████████| 13100/13100 [00:43<00:00, 303.39it/s]

spn in phoneseq Interval(3.49, 3.96, "currys") ['spn']
spn in phoneseq Interval(0.0, 0.48, "carro") ['spn']
spn in phoneseq Interval(1.61, 2.33, "oswalds") ['spn']
spn in phoneseq Interval(0.0, 0.34, "whaley") ['spn']
spn in phoneseq Interval(1.56, 2.14, "oswalds") ['spn']
spn in phoneseq Interval(5.9, 6.84, "babylonians") ['spn']
spn in phoneseq Interval(0.0, 0.39, "misess") ['spn']
spn in phoneseq Interval(1.4, 2.11, "fleetingly") ['spn']
spn in phoneseq Interval(3.3, 3.95, "joshuas") ['spn']


In [67]:
print(len(mfa_symbol_set))
", ".join(sorted(mfa_symbol_set))

82


'a, aj, aw, aː, b, bʲ, c, cʰ, d, dʒ, dʲ, d̪, e, ej, f, fʲ, h, i, iː, j, k, kʰ, l, m, mʲ, m̩, n, n̩, o, ow, p, pʰ, pʲ, s, spn, t, tʃ, tʰ, tʲ, t̪, u, uː, v, vʲ, w, z, æ, ç, ð, ŋ, ɐ, ɑ, ɑː, ɒ, ɒː, ɔ, ɔj, ə, əw, ɚ, ɛ, ɛː, ɜ, ɜː, ɝ, ɟ, ɡ, ɪ, ɫ, ɫ̩, ɱ, ɲ, ɹ, ɾ, ʃ, ʉ, ʉː, ʊ, ʎ, ʒ, ʔ, θ'

# save in format for fastpitch

# save in format for speechbrain 

In [68]:
def parse_textgrid(tier, speechreps, ignore_all_pauses=True):
    # latest MFA replaces silence phones with "" in output TextGrids
    total_num_frames = speechreps.size(0)
    sil_phones = ["sil", "sp", "spn", ""]
    utt_start_time = tier[0].start_time
    utt_end_time = tier[-1].end_time
    words = []
    word_durations = [] 
    sil_durations = []
    start_frames = []
    end_frames = []
    for i, t in enumerate(tier._objects):
        start, end, token = t.start_time, t.end_time, t.text
        start_idx, end_idx, num_frames = extract_reprs_with_timestamps(total_num_frames, start, end, utt_end_time)
    
        if token not in sil_phones:
            words.append(token)
            start_frames.append(start_idx)
            end_frames.append(end_idx)
            word_durations.append(num_frames)
        else:
            sil_durations.append(num_frames)

    return words, word_durations, sil_durations, start_frames, end_frames, utt_start_time, utt_end_time

words, word_durs, sil_durations, start_frames, end_frames, utt_start, utt_end = parse_textgrid(
    tg.get_tier_by_name('words'), reps, ignore_all_pauses=True
)

NameError: name 'tg' is not defined